In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
link = '/content/drive/MyDrive/UFC_mestrado/Sigaa_UFC/1_semestre/aprendizagem_automatica/lista_03_ama/dados/kc2.csv'

In [ ]:
# Bibliotecas

# tratamento de dados
import pandas as pd
import numpy as np

# Calculos matemática e estatística
from scipy import stats

# Tratamento de dados para separar treino, teste e normalização
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

# Bibliotecas
from sklearn import tree

# Gráficos
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("Accent")
sns.set_style("darkgrid")
import plotly.express as px
import plotly.graph_objects as g



# definindo uma semente goblal
np.random.seed(42)

In [ ]:
df = pd.read_csv(link, header=None)

In [ ]:
display(df.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,0.0
1,39.0,4.0,1.0,2.0,105.0,520.19,0.07,13.89,37.44,7227.91,...,29.0,1.0,4.0,2.0,12.0,19.0,61.0,44.0,7.0,0.0
2,1.0,1.0,1.0,1.0,6.0,15.51,0.40,2.50,6.20,38.77,...,0.0,0.0,0.0,0.0,5.0,1.0,5.0,1.0,1.0,0.0
3,15.0,1.0,1.0,1.0,55.0,224.81,0.17,5.73,39.25,1287.55,...,12.0,0.0,1.0,0.0,6.0,11.0,34.0,21.0,1.0,0.0
4,12.0,2.0,1.0,2.0,15.0,45.00,0.17,6.00,7.50,270.00,...,8.0,1.0,0.0,0.0,6.0,2.0,11.0,4.0,3.0,0.0


In [ ]:
# Definindo o dados de entrada e dados de saída.
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

##**Normalizando os Dados**

In [ ]:
# Normalizando os dados
scaler = MinMaxScaler()
X_normalizando = scaler.fit_transform(X)

# **Funções:**

aqui contém funções:                
- calcular desvio padrão            
- calcular a média                   
- calcular devio padrão e média por classes                  

**Class de métricas**               
contém os seguintes métodos:    
- Acurácia                 
- Precisão               
- f1_score                 
- recall  
- especificidade
- formatação das métricas

**Funções de Gráficos:**
- Gráfico de rosca para métricas
- Gráfico para matriz de confusão


In [ ]:
#____________________________________________________________________________________________________________________________
def calcular_desvio_padrao(x):
    desvios = np.sqrt(np.mean((x - np.mean(x, axis=0))**2, axis=0))
    return desvios

#____________________________________________________________________________________________________________________________

def calcular_media(x):
  media = np.mean(x, axis=0)
  return media

#____________________________________________________________________________________________________________________________

# Esta função tem como objetivo de calcular ao desvio padrão e a média de cada classe a partir da acurácia

def desvio_padrao_media_por_classe(scores_treino, scores_teste):

  # Calculando a média das pontuações de validação
  mean_score_treino = calcular_media(scores_treino)
  desvio_score_treino = calcular_desvio_padrao(scores_treino)

  print(f"Acurácia dos 10 folds de teste: ",[f'{score * 100:.2f}%' for score in scores_treino])
  print(f"Média global da Acurácia de validação dos dados de teste: {mean_score_treino * 100:.2f}%")
  print(f"Desvio Padrão global da Acurácia de validação dos dados de teste: {desvio_score_treino * 100:.2f}%")

  print('-'*50)
  grafico_rosca('Média Acuracia',mean_score_treino)

In [ ]:
# Funções de métricas

# Esta classe funciona somente para classificação binária ou seja em que tem dois valores não se aplica a multiclass a partir de três.
class Metricas:

  def __init__(self,real,previsto):

     self.real = real
     self.previsto = previsto

     # Total
     self._total = len(self.real)

     # Verdadeiros Positivos (VP): São as instâncias que são positivas e que o modelo classificou corretamente como positivas.
     self._VP = np.sum((self.real == 1) & (self.previsto == 1))
     # Verdadeiros Negativos (VN): São as instâncias que são negativas e que o modelo classificou corretamente como negativas.
     self._VN = np.sum((self.real == 0) & (self.previsto == 0))
    # Falsos Positivos (FP): São as instâncias que são negativas, mas que o modelo classificou erroneamente como positivas.
     self._FP = np.sum((self.real == 0) & (self.previsto == 1))
    # Falsos Negativos (FN): São as instâncias que são positivas, mas que o modelo classificou erroneamente como negativas.
     self._FN = np.sum((self.real == 1) & (self.previsto == 0))

  def acuracia(self):
     acuracia = (self._VP + self._VN) / (self._VP + self._VN + self._FP + self._FN)
     return acuracia

  def precisao(self):
    precision = self._VP / (self._VP + self._FP)
    return precision

  def recall(self):
    recall = self._VP / (self._VP + self._FN)
    return recall

  def especificidade(self):
    taxa = self._VN / (self._VN + self._FP)
    return taxa

  def f1_score(self):
    score = 2 * ((self.precisao() * self.recall()) / (self.precisao() + self.recall()))
    return score

  def formatacao(self):
    print(f"Acuracia: {self.acuracia() * 100:.2f}%\
    \n especificidade: {self.especificidade() * 100:.2f}%\
    \n recall: {self.recall() * 100:.2f}%\
    \n precisao: {self.precisao() * 100:.2f}%\
    \n f1_score: {self.f1_score() * 100:.2f}%")


In [ ]:
# me = Metricas(y_teste, y_previsto_entropia)
# me.relatorio(numeros_kfold)

# **Gráficos**

In [ ]:
def grafico_linha(titulo, categorias, valores, exibir=True):

    valores = pd.Series(valores)
    desvio_padrao = valores.std().round(2)
    media = valores.mean().round(2)
    mediana = valores.median().round(2)
    erro = valores.sem().round(2)  # Erro padrão médio

    # Definindo o nível de confiança (95% neste caso)
    confidence_level = 0.95
    graus_de_liberdade = len(valores) - 1

    t_critical = stats.t.ppf(q=(1 + confidence_level) / 2, df=graus_de_liberdade) # Calcular o valor crítico
    margin_of_error = t_critical * erro # Calcular a margem de erro
    confidence_interval = (media - margin_of_error).round(2), (media + margin_of_error).round(2) # Calcular o intervalo de confiança


    # Criando o gráfico de linhas com Plotly Express
    fig = px.line(x=categorias, y=valores, title=titulo, text=valores.round(2))
    fig.update_traces(mode='lines+markers+text', textposition='top center')

    # Personalizando o layout do gráfico com Plotly Graph Objects (opcional)
    fig.update_layout(
        xaxis_title='Categorias',
        yaxis_title='Valores',
        hovermode='x',
        template='plotly_white',  # Tema do gráfico (opcional)
        width=1300,  # Largura do gráfico
        height=400,  # Altura do gráfico
        margin=dict(l=20, r=260, t=35, b=20),  # Define as margens
        title_font=dict(size=20, color='black'),  # Estilo do título
        xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),  # Estilo do eixo X
        yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))   # Estilo do eixo Y
    )

    if exibir == True:
        # Adicionar texto ao lado do gráfico
        fig.add_annotation(
            text=f'<b>Resultados</b><br> Desvio Padrão: {desvio_padrao}<br> Média: {media}<br> Mediana: {mediana}\
                  <br> Erro padrão médio: {erro}<br> Margin de erro: {margin_of_error.round(3)}<br> Intervalo de confiança: {confidence_interval}',
            xref='paper',  # Posicionamento em relação ao papel (gráfico)
            yref='paper',  # Posicionamento em relação ao papel (gráfico)
            x=1.25,  # Posição horizontal do texto
            y=0.85,   # Posição vertical do texto
            showarrow=False,  # Não exibir seta
            font=dict(size=14, color='black'),  # Tamanho da fonte do texto
            bgcolor='lightblue',  # Cor de fundo azul claro
            borderwidth=1,  # Largura da borda
            align='left',  # Justifica o texto à esquerda

        )

    return fig.show()

#**Modelo Árvore de decisão**

**Aqui contém funções como:**
- calcular indice Gini
- calcular indice Entropia
- calcular ordenação da árvore de decisão
- calcular a predição e exibir metricas por relatórios e gráficos

In [ ]:
def arvore_decisao(X, y, max_depth=None, min_samples_split=2, indi_impureza='gini'):
    # Função para calcular a entropia
    def calcular_entropia(y):
        classes, contagens = np.unique(y, return_counts=True)  # calcula as classes e suas contagens
        probabilidades = contagens / len(y)  # probabilidades de cada classe
        entropia = -np.sum(probabilidades * np.log2(probabilidades))  # calcula a entropia
        return entropia

    # Função para calcular o índice de Gini
    def calcular_gini(y):
        classes, contagens = np.unique(y, return_counts=True)  # calcula as classes e suas contagens
        probabilidades = contagens / len(y)  # probabilidades de cada classe
        gini = 1 - np.sum(probabilidades ** 2)  # calcula o índice de Gini
        return gini

    # Função para calcular o ganho de informação
    def calcular_ganho_informacao(y, y_esq, y_dir):
        p_esq = len(y_esq) / len(y)  # proporção de dados no lado esquerdo
        p_dir = len(y_dir) / len(y)  # proporção de dados no lado direito

        if indi_impureza == 'entropia':
            ganho_informacao = calcular_entropia(y) - (p_esq * calcular_entropia(y_esq) + p_dir * calcular_entropia(y_dir))
            return ganho_informacao

        elif indi_impureza == 'gini':
            ganho_informacao = calcular_gini(y) - (p_esq * calcular_gini(y_esq) + p_dir * calcular_gini(y_dir))
            return ganho_informacao

    # Função para dividir o conjunto de dados com base em um limiar
    def dividir_conjunto(X, y, indice_caracteristica, limiar):
        mascara_esq = X[:, indice_caracteristica] <= limiar  # máscara para o lado esquerdo
        mascara_dir = ~mascara_esq  # máscara para o lado direito
        return X[mascara_esq], y[mascara_esq], X[mascara_dir], y[mascara_dir]

    # Função para encontrar a melhor divisão
    def encontrar_melhor_divisao(X, y):
        melhor_ganho = -1
        melhor_indice_caracteristica = -1
        melhor_limiar = None
        n_amostras, n_caracteristicas = X.shape

        for indice_caracteristica in range(n_caracteristicas):
            limiares = np.unique(X[:, indice_caracteristica])  # possíveis limiares para a característica
            for limiar in limiares:
                X_esq, y_esq, X_dir, y_dir = dividir_conjunto(X, y, indice_caracteristica, limiar)
                if len(y_esq) > 0 and len(y_dir) > 0:
                    ganho = calcular_ganho_informacao(y, y_esq, y_dir)
                    if ganho > melhor_ganho:
                        melhor_ganho = ganho
                        melhor_indice_caracteristica = indice_caracteristica
                        melhor_limiar = limiar

        return melhor_indice_caracteristica, melhor_limiar

    # Função para construir a árvore de decisão
    def construir_arvore(X, y, profundidade=0):
        n_amostras, n_caracteristicas = X.shape
        if n_amostras >= min_samples_split and (max_depth is None or profundidade < max_depth):
            indice_caracteristica, limiar = encontrar_melhor_divisao(X, y)
            if indice_caracteristica is not None:
                X_esq, y_esq, X_dir, y_dir = dividir_conjunto(X, y, indice_caracteristica, limiar)
                subarvore_esq = construir_arvore(X_esq, y_esq, profundidade + 1)
                subarvore_dir = construir_arvore(X_dir, y_dir, profundidade + 1)
                return {'indice_caracteristica': indice_caracteristica, 'limiar': limiar, 'esquerda': subarvore_esq, 'direita': subarvore_dir}

        # Convertendo y para inteiros antes de usar np.bincount
        valor_folha = np.argmax(np.bincount(y.astype(int)))
        return valor_folha

    arvore = construir_arvore(X, y)
    return arvore

# Função para fazer previsões usando a árvore de decisão construída
def predicao(arvore, X):
    def prever_amostra(no, x):
        if isinstance(no, dict):
            if x[no['indice_caracteristica']] <= no['limiar']:
                return prever_amostra(no['esquerda'], x)
            else:
                return prever_amostra(no['direita'], x)
        else:
            return no

    previsoes = [prever_amostra(arvore, x) for x in X]
    return np.array(previsoes)

***Escolha:***             
criterio:   
- grid     
- entropy  


Arvore de decisão

In [ ]:

acuracias_gini = []
revocação_gini = []
precisao_gini = []
f1_score_gini = []

# Lista para armazenar as pontuações de validação de cada fold
numeros_kfold = []

salvando_y_previsto_de_cada_fold = []


# Definindo a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state = 42)

# Preparando o modelo
# arvore_decisao = tree.DecisionTreeClassifier(criterion = 'gini', max_depth = 4)



# Loop sobre os folds
for fold_idx, (treino_index, teste_index) in enumerate(kf.split(X_normalizando), start=1):
    X_treino, X_teste = X_normalizando[treino_index], X_normalizando[teste_index]
    y_treino, y_teste = np.array(y[treino_index]), np.array(y[teste_index])

    # arvore = arvore_decisao(X_treino, y_treino)

    #y_previsto = arvore.predict(X_teste)

    arvore = arvore_decisao(X_treino, y_treino, max_depth=3, indi_impureza='gini')
    y_previsto_gini = predicao(arvore, X_teste)

    # Avaliando o modelo
    metrics_01 = Metricas(y_teste, y_previsto_gini)
    acuracia = metrics_01.acuracia()
    revogacao = metrics_01.recall()
    precisao = metrics_01.precisao()
    f1_score = metrics_01.f1_score()

    acuracias_gini.append(acuracia)
    revocação_gini.append(revogacao)
    precisao_gini.append(precisao)
    f1_score_gini.append(f1_score)

    qt = f'Fold: {fold_idx}:'
    numeros_kfold.append(qt)


display('Resultados usando o critério gini')

for lista,numeros in zip(acuracias_gini,numeros_kfold):
  display(f'{numeros} acurácia ->  {lista*100:.2f}%')

'Resultados usando o critério gini'

'Fold: 1: acurácia ->  72.73%'

'Fold: 2: acurácia ->  54.55%'

'Fold: 3: acurácia ->  81.82%'

'Fold: 4: acurácia ->  90.91%'

...

'Fold: 6: acurácia ->  85.71%'

'Fold: 7: acurácia ->  71.43%'

'Fold: 8: acurácia ->  76.19%'

'Fold: 9: acurácia ->  71.43%'

'Fold: 10: acurácia ->  71.43%'

In [ ]:
rel = pd.DataFrame(index = numeros_kfold, columns=['Acuracia', 'Precisao', 'Recall', 'F1_score'])
rel['Acuracia'] = acuracias_gini
rel['Precisao'] = precisao_gini
rel['Recall'] = revocação_gini
rel['F1_score'] = f1_score_gini

In [ ]:
cabecalho = {
    'selector': 'th',
    'props': 'font-family: Helvetica; color: #dddd55; background-color: #34495E; text-align: center;'
}

celulas = {
    'selector': 'td',
    'props': 'font-family: Helvetica; color: white; background-color: #34495E; text-align: left;'
}


estilo_01 = rel.style.format('{:.2f}')
estilo_01.set_table_styles([cabecalho, celulas])

,Acuracia,Precisao,Recall,F1_score
Fold: 1:,0.73,0.77,0.77,0.77
Fold: 2:,0.55,0.50,0.70,0.58
Fold: 3:,0.82,0.90,0.75,0.82
Fold: 4:,0.91,0.82,1.00,0.90
Fold: 5:,0.81,0.78,0.78,0.78
Fold: 6:,0.86,0.87,0.93,0.90
Fold: 7:,0.71,0.67,0.80,0.73
Fold: 8:,0.76,0.83,0.56,0.67
Fold: 9:,0.71,0.88,0.58,0.70
Fold: 10:,0.71,0.64,0.78,0.70


In [ ]:
grafico_linha('Árvore de decisão gini Acurácia', numeros_kfold, acuracias_gini, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão gini Recall', numeros_kfold, revocação_gini, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão gini Precisão', numeros_kfold, precisao_gini, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão gini F1_score', numeros_kfold, f1_score_gini, exibir=True)

In [ ]:
acuracias_entropia = []
recall_entropia = []
precisao_entropia = []
f1_score_entropia = []

numeros_kfold = []

salvando_y_previsto_de_cada_fold = []


# Loop sobre os folds
for fold_idx, (treino_index, teste_index) in enumerate(kf.split(X_normalizando), start=1):
    X_treino, X_teste = X_normalizando[treino_index], X_normalizando[teste_index]
    y_treino, y_teste = np.array(y[treino_index]), np.array(y[teste_index])


    arvore = arvore_decisao(X_treino, y_treino, max_depth=4,indi_impureza='entropia')
    y_previsto_entropia = predicao(arvore, X_teste)



    # Avaliando o modelo
    metrics_02 = Metricas(y_teste, y_previsto_entropia)
    acuracia = metrics_02.acuracia()
    recall = metrics_02.recall()
    precisao = metrics_02.precisao()
    f1_score = metrics_02.f1_score()

    acuracias_entropia.append(acuracia)
    recall_entropia.append(recall)
    precisao_entropia.append(precisao)
    f1_score_entropia.append(f1_score)

    qt = f'Fold: {fold_idx}:'
    numeros_kfold.append(qt)


display('Resultados usando o critério gini')

for lista,numeros in zip(acuracias_entropia,numeros_kfold):
  display(f'{numeros} acurácia ->  {lista*100:.2f}%')

'Resultados usando o critério gini'

'Fold: 1: acurácia ->  63.64%'

'Fold: 2: acurácia ->  59.09%'

'Fold: 3: acurácia ->  86.36%'

'Fold: 4: acurácia ->  81.82%'

...

'Fold: 6: acurácia ->  71.43%'

'Fold: 7: acurácia ->  80.95%'

'Fold: 8: acurácia ->  80.95%'

'Fold: 9: acurácia ->  71.43%'

'Fold: 10: acurácia ->  76.19%'

In [ ]:
rel_02 = pd.DataFrame(index = numeros_kfold, columns=['Acuracia', 'Precisao', 'Recall', 'F1_score'])
rel_02['Acuracia'] = acuracias_entropia
rel_02['Precisao'] = precisao_entropia
rel_02['Recall'] = recall_entropia
rel_02['F1_score'] = f1_score_entropia

In [ ]:

estilo_02 = rel_02.style.format('{:.2f}')
estilo_02.set_table_styles([cabecalho, celulas])

,Acuracia,Precisao,Recall,F1_score
Fold: 1:,0.64,0.78,0.54,0.64
Fold: 2:,0.59,0.54,0.70,0.61
Fold: 3:,0.86,1.00,0.75,0.86
Fold: 4:,0.82,0.86,0.67,0.75
Fold: 5:,0.76,0.70,0.78,0.74
Fold: 6:,0.71,0.83,0.71,0.77
Fold: 7:,0.81,0.75,0.90,0.82
Fold: 8:,0.81,0.86,0.67,0.75
Fold: 9:,0.71,0.88,0.58,0.70
Fold: 10:,0.76,0.67,0.89,0.76


In [ ]:
grafico_linha('Árvore de decisão com entropia Acurácia', numeros_kfold,acuracias_entropia, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão com entropia Precisao', numeros_kfold, precisao_entropia, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão com entropia Recall', numeros_kfold, recall_entropia, exibir=True)

In [ ]:
grafico_linha('Árvore de decisão com entropia F1_score', numeros_kfold, f1_score_entropia, exibir=True)